## 🔐 Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login
```

or

```bash
az login --use-device-code
```

# 🔄 Azure AI Agent with Existing Agent - Persistent Financial Advisor 🏦

This notebook demonstrates working with pre-existing Azure AI Agents using `AzureAIProjectAgentProvider` for production scenarios where you want to reuse a persistent Advisor agent.

## Features Covered:
- Creating persistent Azure AI agents for banking services
- Working with existing agent IDs using `provider.get_agent()`
- Agent lifecycle management (create, use, delete)
- Production patterns for reusing Financial Advisor agents
- Manual cleanup of agents

### ⚠️ Important Note ⚠️
> **Persistent agents are useful for production scenarios where you want consistent behavior across sessions. The agent retains its configuration and can be accessed by ID.**

## Prerequisites

Before running this notebook, ensure you have:

1. **Azure AI Project**: Access to an Microsoft Foundry project with deployed models
2. **Authentication**: Azure CLI installed and authenticated (`az login --use-device-code`)
3. **Environment Variables**: Set up your `.env` file with:
   - `AI_FOUNDRY_PROJECT_ENDPOINT`
   - `AZURE_AI_MODEL_DEPLOYMENT_NAME`
4. **Dependencies**: Required agent-framework packages installed

If you need to use a different tenant:
```bash
az login --tenant <tenant-id>
```

This example demonstrates how to work with an existing Azure AI agent by connecting to it using its agent ID.

## Import Libraries

Import the required libraries using the NEW `AzureAIProjectAgentProvider` API:

In [ ]:
import os
from pathlib import Path
from random import randint, uniform
from typing import Annotated

from agent_framework.azure import AzureAIAgentsProvider
from azure.ai.agents.aio import AgentsClient
from azure.identity.aio import AzureCliCredential
from pydantic import Field

## Initial Setup

Load environment variables from the `.env` file:

In [ ]:
from pathlib import Path
import os
from dotenv import load_dotenv

# Load environment variables
notebook_path = Path().absolute()
load_dotenv('../../.env')

print("📝 Note: This notebook demonstrates working with existing (persistent) agents")
print("✅ Environment variables and async imports are ready for use")

## Check Environment Variables

Verify that the required environment variables are set:

In [ ]:
# Check required environment variables
required_vars = ["AI_FOUNDRY_PROJECT_ENDPOINT", "AZURE_AI_MODEL_DEPLOYMENT_NAME"]
missing_vars = []

for var in required_vars:
    value = os.getenv(var)
    if value:
        print(f"✅ {var}: {value[:50]}..." if len(str(value)) > 50 else f"✅ {var}: {value}")
    else:
        print(f"❌ {var}: Not set")
        missing_vars.append(var)

if missing_vars:
    print(f"\n⚠️  Please set the following environment variables: {', '.join(missing_vars)}")
else:
    print("\n✅ All required environment variables are set!")

## Define Function Tools 🏦

Let's define banking functions that our Financial Advisor agent can use:

In [ ]:
def get_account_balance(
    account_id: Annotated[str, Field(description="The customer account ID to check balance for.")],
) -> str:
    """Get the current balance for a customer account."""
    balance = round(uniform(5000, 75000), 2)
    return f"Account {account_id}: Current balance is ${balance:,.2f}"


def get_loan_rates(
    loan_type: Annotated[str, Field(description="Type of loan: mortgage, auto, personal, or business")],
) -> str:
    """Get current interest rates for different loan types."""
    rates = {
        "mortgage": {"rate": 6.5, "term": "30 years"},
        "auto": {"rate": 7.2, "term": "5 years"},
        "personal": {"rate": 10.5, "term": "3 years"},
        "business": {"rate": 8.0, "term": "10 years"}
    }
    loan_type = loan_type.lower()
    if loan_type in rates:
        return f"Current {loan_type} loan rate: {rates[loan_type]['rate']}% APR, typical term: {rates[loan_type]['term']}"
    return f"Unknown loan type. Available: mortgage, auto, personal, business"

## Create and Use Existing Agent 🔄

This example shows how to:
1. Create a persistent Financial Advisor agent using Azure SDK directly
2. Use `provider.get_agent()` to wrap an existing agent
3. Properly clean up resources

**Key API Method**: `provider.get_agent(name=...)` retrieves an existing agent without creating a new one.

In [ ]:
async def main():
    """Demonstrate using provider.as_agent() to work with an existing Azure AI Agent.
    
    This function shows the complete workflow:
    1. Create an agent using the AgentsClient
    2. Use provider.as_agent() to wrap it with framework features
    3. Add custom function tools for banking operations
    """
    print("=== 🔄 Azure AI Agent with Existing Agent ===")
    
    async with (
        AzureCliCredential() as credential,
        AgentsClient(
            endpoint=os.environ["AI_FOUNDRY_PROJECT_ENDPOINT"],
            credential=credential
        ) as agents_client,
        AzureAIAgentsProvider(agents_client=agents_client) as provider,
    ):
        # Step 1: Create an agent on the service with default instructions
        # These instructions will persist on created agent for every run.
        azure_ai_agent = await agents_client.create_agent(
            model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"],
            instructions="You are a helpful Financial Advisor for a retail bank. Help customers with account inquiries and loan information.",
        )
        
        print(f"✅ Created agent: {azure_ai_agent.id}")

        try:
            # Step 2: Wrap existing agent instance using provider.as_agent()
            agent = provider.as_agent(
                azure_ai_agent,
                tools=[get_account_balance, get_loan_rates],  # Add tools to the wrapped agent
            )
            
            print(f"✅ Wrapped agent with framework features")
            
            # Use the agent for banking queries
            query = "What's the balance for account ACC-PROD-001 and what are the current mortgage rates?"
            print(f"\n🤔 Customer: {query}")
            result = await agent.run(query)
            print(f"🏦 Advisor: {result}")
            
        finally:
            # Commented out to keep the agent for future use
            # To delete the agent later, uncomment the following lines:
            # await agents_client.delete_agent(azure_ai_agent.id)
            # print(f"\n🗑️ Deleted agent: {azure_ai_agent.id}")
            pass

## Execute the Example 🚀

Run the main function to see the existing agent workflow:

In [ ]:
# Run the main function
await main()

## Key Takeaways 📚







### Key Methods for Existing Agents

```python
# Create agent using SDK directly
created_agent = await client.agents.create_agent(model=..., name=..., instructions=...)

# Wrap existing agent with provider
async with AzureAIProjectAgentProvider(credential=credential) as provider:
    agent = await provider.get_agent(
        name="AgentName",        # or agent_id="asst_..."
        tools=[function1, function2],  # Add tools to the wrapped agent
    )
```

### Production Patterns

1. **Persistent Agents**: Create once, use across multiple sessions
2. **Portal Integration**: Use agents created in Azure AI Studio
3. **Tool Injection**: Add tools when retrieving existing agents
4. **Lifecycle Management**: Track agent IDs for cleanup

### Use Cases

- **Persistent Financial Advisors**: Same agent persona for all customers
- **Branch-Specific Agents**: Different agents for different bank branches
- **Specialized Advisors**: Mortgage, Investment, Personal Banking agents
- **Session Continuity**: Maintain context across customer interactions

### Best Practices

1. **Agent Naming**: Use clear, descriptive names for easy retrieval
2. **ID Tracking**: Store agent IDs for cleanup and monitoring
3. **Error Handling**: Handle cases where agent doesn't exist
4. **Resource Cleanup**: Delete unused agents to manage costs
5. **Tool Management**: Add/update tools when retrieving agents
